In [1]:
import pandas as pd
import numpy as np
import helper_functions as sc

from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [7]:
data_ready = pd.read_csv('data\\bundesliga_2017_ready.csv', index_col=0)

X_train = data_ready.drop(['target', 'round'], axis= 1)
y_train = data_ready['target']
features = list(X_train.columns)

# Normalization
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = features
X_train.head()

C:\Users\kr_fl\anaconda\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,goal_difference_1,form_1,form_weighted_1,goal_difference_2,form_2,form_weighted_2,home_away,team_1_B,team_1_C,team_1_D,team_2_B,team_2_C,team_2_D
0,0.362745,1.0,1.00,0.323529,0.0,0.00,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.323529,0.0,0.00,0.362745,1.0,1.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.352941,1.0,0.75,0.333333,0.0,0.00,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.333333,0.0,0.00,0.352941,1.0,0.75,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.333333,0.0,0.00,0.352941,1.0,0.00,1.0,0.0,1.0,0.0,1.0,0.0,0.0


## train models on last season

In [3]:
# ridge regression
mdl_ridge = LogisticRegression(C=1, penalty='l2', solver='liblinear', max_iter=100, multi_class = 'auto') # Ridge
mdl_ridge.fit(X_train,y_train)

# knn
mdl_knn = KNeighborsClassifier(n_neighbors=29)  
mdl_knn.fit(X_train,y_train)

# randomforest
mdl_rf = RandomForestClassifier(min_samples_leaf=5, random_state=1, n_jobs=-1,
                                n_estimators=1000, max_depth=5)
mdl_rf.fit(X_train,y_train)

# neural network
mdl_mlp = MLPClassifier(hidden_layer_sizes = (10, ), alpha = 10,
                        activation='relu', solver='lbfgs',
                        batch_size=20, learning_rate='invscaling', learning_rate_init=0.001)
mdl_mlp.fit(X_train,y_train)


# ensemble
X_ensemble = sc.get_ensemble_x(X_train, mdl_ridge, mdl_knn, mdl_rf, mdl_mlp)

# train ensemble model
mdl_lasso = LogisticRegression(C = 10, penalty='l1', solver='liblinear',
                               max_iter=100, multi_class = 'auto')
mdl_lasso.fit(X_ensemble, y_train)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

## Simulation of next season

In [4]:
# load new season data
data_new = pd.read_csv('data\\bundesliga_2018_ready.csv', index_col=0)
features.append('round')
features.append('target')
data_new = data_new[features]
data_new.head()

,goal_difference_1,form_1,form_weighted_1,goal_difference_2,form_2,form_weighted_2,home_away,team_1_B,team_1_C,team_1_D,team_2_B,team_2_C,team_2_D,round,target
0,2,3.0,12.0,-2,0.0,0.0,1,0,0,0,0,1,0,Regular Season - 1,3
1,-2,0.0,0.0,2,3.0,0.0,1,0,1,0,0,0,0,Regular Season - 1,3
2,-1,0.0,0.0,1,3.0,0.0,1,0,1,0,1,0,0,Regular Season - 1,0
3,1,3.0,9.0,-1,0.0,0.0,1,1,0,0,0,1,0,Regular Season - 1,0
4,1,3.0,9.0,-1,0.0,0.0,1,1,0,0,0,1,0,Regular Season - 1,3


In [9]:
# simulation
results = pd.DataFrame()

print('starting simulation')

for gameday in data_new['round'].unique():
    gameday_data = data_new[data_new['round'] == gameday]
    
    X = gameday_data.drop(['target', 'round'], axis= 1)
    y = gameday_data['target']
    features = list(X.columns)
    
    # Normalization
    X = scaler.transform(X)
    X = pd.DataFrame(X)
    X.columns = features
    
    X_ensemble = sc.get_ensemble_x(X, mdl_ridge, mdl_knn, mdl_rf, mdl_mlp)
    
    y_hat = mdl_lasso.predict(X_ensemble)
    y_hat_proba = mdl_lasso.predict_proba(X_ensemble)
    bet = [any(y_hat_proba[i] > 0.8) for i in range(len(y_hat_proba))]

    results = results.append(pd.DataFrame({'gameday': list(np.repeat(gameday, len(y))),
                                           'y':y,
                                           'y_hat':y_hat,
                                           'bet': bet}))
    
    # retrain models
    X_train = X_train.append(X)
    y_train = y_train.append(y)
    
    mdl_ridge.fit(X_train, y_train)
    mdl_knn.fit(X_train, y_train)
    mdl_rf.fit(X_train, y_train)
    mdl_mlp.fit(X_train, y_train)
    
    X_ensemble = sc.get_ensemble_x(X_train, mdl_ridge, mdl_knn, mdl_rf, mdl_mlp)
    
    mdl_lasso.fit(X_ensemble, y_train)
    
    print('finished simulating gameday: ', gameday)

starting simulation
finished simulating gameday:  Regular Season - 1
finished simulating gameday:  Regular Season - 2
finished simulating gameday:  Regular Season - 3
finished simulating gameday:  Regular Season - 4
finished simulating gameday:  Regular Season - 5
finished simulating gameday:  Regular Season - 6
finished simulating gameday:  Regular Season - 7
finished simulating gameday:  Regular Season - 8
finished simulating gameday:  Regular Season - 9
finished simulating gameday:  Regular Season - 10
finished simulating gameday:  Regular Season - 11
finished simulating gameday:  Regular Season - 12
finished simulating gameday:  Regular Season - 13
finished simulating gameday:  Regular Season - 14
finished simulating gameday:  Regular Season - 15
finished simulating gameday:  Regular Season - 16
finished simulating gameday:  Regular Season - 17
finished simulating gameday:  Regular Season - 18
finished simulating gameday:  Regular Season - 19
finished simulating gameday:  Regular S

## Accuracy 

In [10]:
print('final accuracy of ensemble on test set =' , round(accuracy_score(results['y'], results['y_hat']), 4))
print('\n confusion matrix: \n', confusion_matrix(results['y'], results['y_hat']))  
print('\n', classification_report(results['y'], results['y_hat'])) 

results_bet = results[results['bet'] == True]
print('final accuracy of ensemble on test set with threshold = ',
      round(accuracy_score(results_bet['y'], results_bet['y_hat']), 4))
print('number of bets = ', len(results_bet), '\n max bets: ', len(results))

final accuracy of ensemble on test set = 0.4003

 confusion matrix: 
 [[ 64  30  96]
 [ 34  41  71]
 [ 91  45 140]]

               precision    recall  f1-score   support

           0       0.34      0.34      0.34       190
           1       0.35      0.28      0.31       146
           3       0.46      0.51      0.48       276

   micro avg       0.40      0.40      0.40       612
   macro avg       0.38      0.37      0.38       612
weighted avg       0.40      0.40      0.40       612

final accuracy of ensemble on test set with threshold =  0.4574
number of bets =  129 
 max bets:  612
